Imports

In [ ]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
#controllo il massimo valore di altezza e larghezza per fare il padding
import cv2
import os

# Directory containing the images
data_dir = 'Dataset'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape
        if(height > 1000 or width > 1000):
            print(filename)
            print(image.shape)

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

        

#Image with different max_height and max_width: Queretaro-TEC_parcheggi.png
print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

Store canny edge

In [ ]:
dataset_path = 'Dataset'
edge_path = 'Dataset_edge'
def get_edge_image(image, low_threshold, high_threshold):
    # Convert the image to grayscale if it's not already
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    return edges
#edge detection + padding immagini
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)
        # Ottieni l'immagine del bordo
        edge_image = get_edge_image(image, low_threshold=250, high_threshold=350)
        
        # Apply padding to the edge image
        max_height = 2108
        max_width = 2113
        padding_height = max(0, max_height - edge_image.shape[0])
        padding_width = max(0, max_width - edge_image.shape[1])
        edge_image_padded = cv2.copyMakeBorder(edge_image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        # Verificare se il nome del file contiene "_testing" o "_training"
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(edge_path, new_filename)
            cv2.imwrite(output_image_path, edge_image_padded)

print("Il rilevamento dei bordi è stato applicato a tutte le immagini nella cartella.")


In [ ]:
import os
import shutil

# Define the paths to the mask and image folders
mask_folder = 'Dataset_mask'
image_folder = 'Dataset_edge'

# Create a new folder to store the combined images and masks
combined_folder = 'Dataset_combined'
os.makedirs(combined_folder, exist_ok=True)

# Copy all files from the mask folder to the combined folder
for mask_file in os.listdir(mask_folder):
    shutil.copy2(os.path.join(mask_folder, mask_file), combined_folder)

# Copy all files from the image folder to the combined folder
for image_file in os.listdir(image_folder):
    shutil.copy2(os.path.join(image_folder, image_file), combined_folder)

print("Masks and images have been combined and stored in the 'Combined' folder.")

Dataset preparation

DATASET CLASS

In [ ]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                for mask_path in self.mask_paths:
                    mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
                    if image_filename in mask_filename:
                        self.pairs.append((image_path, mask_path))
                        break  # Se trova una corrispondenza, passa al prossimo file immagine
        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
            print(image_array.shape)
        

    


        return image_array, masks
        


DATASET SLPIT

In [ ]:
#controllo il massimo valore di altezza e larghezza per fare il padding
import cv2
import os

# Directory containing the images
data_dir = 'Dataset_combined'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

        # Check if the image has different max_height and max_width
        if height != 2108 or width != 2113:
            print("Image with different max_height and max_width:", filename)

#Image with different max_height and max_width: Queretaro-TEC_parcheggi.png
print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

In [ ]:
import random
from torch.utils.data import random_split
from torchvision.transforms import functional as F

# magari rifare il dataloader con due cartelle
image_path = 'Dataset_combined'

target_size = (max_width, max_height)

transform = transforms.Compose([
    
    transforms.ToTensor(),
    # Add other transforms here as needed
])


# Create the dataset
dataset = ParkingLotDataset(image_path, transforms=transform)

# Define the proportions for the split
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1


# Shuffle the dataset
random.shuffle(dataset.pairs)

# Calculate the sizes of each split
total_size = len(dataset)
print(total_size)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_pairs = dataset.pairs[:train_size]
val_pairs = dataset.pairs[train_size:train_size+val_size]
test_pairs = dataset.pairs[train_size+val_size:]

# Create datasets for each split
train_dataset = ParkingLotDataset(image_path, pairs=train_pairs, transforms=transform)
val_dataset = ParkingLotDataset(image_path, pairs=val_pairs, transforms=transform)
test_dataset = ParkingLotDataset(image_path, pairs=test_pairs, transforms=transform )

# Now you can create data loaders for each split
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) 


Model

In [ ]:
import torch
import torch.nn as nn
from torchvision.models.segmentation import fcn_resnet50


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define the model
model = fcn_resnet50(pretrained=False, num_classes=2).to(device)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        # Forward pass
        outputs = model(images)['out']
        loss = criterion(outputs, masks)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Train

In [ ]:
# Define the number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model.train()
    for images, targets in train_data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        for images, targets in val_data_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            # You can add here code to compute metrics and print them
